# 안녕하세요^^ 
# AIVLE 미니 프로젝트3차 팀 미션입니다. 
# 주어진 문제를 잘 읽고 해결하시길 바랍니다.


## 올레TV 예능 콘텐츠 중 하나인 프로그램 "코미디빅리그"의 영상을 분석하고 코너를 분류하는 AI 모델을 만들어 보세요.

"코미디빅리그" 는 tvN에서 방영 중인 서바이벌 형식의 공개 코미디 프로그램으로 3개월 단위로 인기 코너를 선정하는 방식인 쿼제를 도입하고 있다. 
이번 과제는 2022년 1쿼터(1~3월) 방영분 중 2월 방영한 3편을 학습 데이터로 하여 3월 방송의 코너 구간을 추출해내는 것입니다.

아래의 조건들을 참고하여 문제를 해결하세요.
> - 주어진 데이터 ( Train 데이터 :  2월 방영분 3편 / Test 데이터 : 3월 방영분 1편 )
> - 제출 파일은 <font color="red">.ipynb 파일</font>과  조에서 각 과정마다 문제 해결을 위해 진행한 방법을 정리한 <font color="red">발표자료(.pptx)</font>를 포함히여 제출합니다.<br> 
①데이터 분석 및 학습데이터 생성, ②데이터 전처리, ③AI 모델링(아키텍처, 성능비교), ④분류 결과 <br> 
※ 각 과정은 문제 해결 방법에 따라 변경 가능<br><br>
> <b>제출 파일( 압축파일명 : [팀]미니프로젝트3차_AI 0반_00조.zip) </b> <br>
>> ① 코드 : [팀]미니프로젝트3차_AI 0반_00조.ipynb <br>
>> ② 발표자료 : [팀]미니프로젝트3차_AI 0반 00조.pptx <br>




---

# 0. 환경 설정하기

In [ ]:
import os
import glob
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

### 1) 구글 드라이브 연결하기

In [ ]:
# 코랩 사용 시 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 2) 경로 확인하기

In [ ]:
# ROOT_PATH 확인 
import os

if os.getcwd() == '/content' :
  # 구글 드라이브 사용 시 
  ROOT_PATH = "/content/drive/MyDrive/AIVLE_3rd[Team]"
else :
  ROOT_PATH = os.path.abspath('..')

# 비디오 파일이 저장되어 있는 폴더
VIDEO_PATH = ROOT_PATH + "/video"

# 모델 학습을 위한 데이터 
TRAIN_PATH = ROOT_PATH + "/train"
# 모델 예측을 위한 test 데이터
TEST_PATH = ROOT_PATH + "/test"

### 3) 영상 데이터 다운로드하기
아래 코드를 실행하세요.

# 미니프로젝트 3차 팀 미션 지금부터 시작하세요!

In [ ]:
# 여기에서부터 입력하세요.








In [ ]:
# 비디오 정보 확인
def video_info(video_path):
    video = cv2.VideoCapture(video_path)
    if video.isOpened():
        video_name   = video_path[video_path.rfind('/') + 1:]
        video_width  = video.get(cv2.CAP_PROP_FRAME_WIDTH)
        video_height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
        video_length = video.get(cv2.CAP_PROP_FRAME_COUNT)
        video_fps    = video.get(cv2.CAP_PROP_FPS)

        print("파일 이름    :", video_name)
        print("가로         :", video_width)
        print("세로         :", video_height)
        print("총 프레임 수 :", video_length)
        print("FPS          :", video_fps)
        print("영상 길이    : %d 초 " %round(video_length/video_fps))
        print("---------------------")
        video.release()
    else:
        print("video open error")
        return -1

In [ ]:
video_info(VIDEO_PATH + '/train/220206.mp4')
video_info(VIDEO_PATH + '/train/220220.mp4')
video_info(VIDEO_PATH + '/test/220320.mp4')

파일 이름    : 220206.mp4
가로         : 854.0
세로         : 480.0
총 프레임 수 : 93479.0
FPS          : 30.0
영상 길이    : 3116 초 
---------------------
파일 이름    : 220220.mp4
가로         : 854.0
세로         : 480.0
총 프레임 수 : 100618.0
FPS          : 30.0
영상 길이    : 3354 초 
---------------------
파일 이름    : 220320.mp4
가로         : 854.0
세로         : 480.0
총 프레임 수 : 95460.0
FPS          : 30.0
영상 길이    : 3182 초 
---------------------


In [ ]:
# 이미지 수집
def save_images(video_path, train_test, point_list):
    video = cv2.VideoCapture(video_path)

    TIME_MEASUERMENT_UNIT = 10

    video_name   = video_path[video_path.rfind('/')+1 : video_path.rfind('.')]

    # if not os.path.exists(IMAGE_PATH):
    #     os.mkdir(IMAGE_PATH)

    count = 0
    i = 0

    while video.isOpened():
        ret,frame = video.read()
        if ret:
            # 현재 프레임 위치 (msec) 
            frame_sec = video.get(cv2.CAP_PROP_POS_MSEC)/1000
            if frame_sec.is_integer():
                if (frame_sec % TIME_MEASUERMENT_UNIT == 0):
                    if round(frame_sec) < point_list[0]:
                        continue
                    elif round(frame_sec) == point_list[0]: 
                        i += 1
                    elif round(frame_sec) > point_list[4]:
                        continue
                    if round(frame_sec) < point_list[i]:
                        if train_test == 'train':
                            IMAGE_PATH = ROOT_PATH + "/image/train/" + str(video_name) + '/' + str(i)
                        elif train_test == 'test':
                            IMAGE_PATH = ROOT_PATH + "/image/test/" + str(video_name) + '/' + str(i)
                        if not os.path.exists(IMAGE_PATH):
                            os.makedirs(IMAGE_PATH)
                        filename = IMAGE_PATH + "/" + str(round(frame_sec)) + ".jpg"
                        cv2.imwrite(filename, frame)
                    else:
                        i += 1
        else:
            break

    print(video_name + "is done")
  

In [ ]:
plist_220206 = [127, 966, 1600, 2333, 3027]
plist_220220 = [124, 840, 1585, 2470, 3254]
plist_220320 = [128, 1040, 1556, 2358, 2949]

In [ ]:
save_images(VIDEO_PATH + '/train/220206.mp4', "train", plist_220206)
save_images(VIDEO_PATH + '/train/220220.mp4', "train", plist_220220)
save_images(VIDEO_PATH + '/train/220320.mp4', "test", plist_220320)

220206is done
220220is done
220320is done
